In [72]:
!pip install -q llama-index

In [73]:
from llama_index.core import SimpleDirectoryReader


In [74]:
documentos = SimpleDirectoryReader(input_dir='documentos')


In [75]:
documentos.input_files

[PosixPath('/content/documentos/doc_info_agricultura.pdf')]

In [76]:
docs = documentos.load_data()

In [77]:
len(docs)

3

In [78]:
print(docs[0].get_metadata_str())

page_label: 1
file_name: doc_info_agricultura.pdf
file_path: /content/documentos/doc_info_agricultura.pdf
file_type: application/pdf
file_size: 30901
creation_date: 2025-04-02
last_modified_date: 2025-04-02


In [79]:
docs[0].__dict__

{'id_': 'eb51a210-3631-49fa-a993-20deefeb17e8',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'doc_info_agricultura.pdf',
  'file_path': '/content/documentos/doc_info_agricultura.pdf',
  'file_type': 'application/pdf',
  'file_size': 30901,
  'creation_date': '2025-04-02',
  'last_modified_date': '2025-04-02'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='Guia de Preparo do Solo para Plantio Informações Gerais P: Qual a importância do preparo adequado do solo? R: O preparo adequado do solo garante que as plantas tenham acesso aos nutrientes necessários, além de melhorar a

In [80]:
from llama_index.core.node_parser import SentenceSplitter
node_parser = SentenceSplitter(chunk_size=1200)

In [81]:
nodes = node_parser.get_nodes_from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [82]:
nodes

[TextNode(id_='26350281-7768-4de1-9b00-1cc7be0d26e1', embedding=None, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': '/content/documentos/doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-04-02', 'last_modified_date': '2025-04-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eb51a210-3631-49fa-a993-20deefeb17e8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': '/content/documentos/doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-04-02', 'last_modified_date': '2025-04-02'}, hash='b58f702a09b7f

In [83]:
len(nodes)

3

In [84]:
nodes

[TextNode(id_='26350281-7768-4de1-9b00-1cc7be0d26e1', embedding=None, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': '/content/documentos/doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-04-02', 'last_modified_date': '2025-04-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eb51a210-3631-49fa-a993-20deefeb17e8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': '/content/documentos/doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-04-02', 'last_modified_date': '2025-04-02'}, hash='b58f702a09b7f

In [85]:
!pip install -q  llama-index-embeddings-huggingface

In [86]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [87]:
# Classe personalizada para integrar ao Chroma DB

class ChromaEmbeddingWrapper:
    def __init__(self, model_name: str):  # inicializa com o modelo embedding
        self.model = HuggingFaceEmbedding(model_name=model_name)

    def __call__(self, input):  # Converte a entrada para o formato compatível com o HuggingFace
        return self.model.embed(input)

In [88]:
embed_model_chroma = ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

In [89]:
# instalação do chroma db
!pip install -q llama-index-vector-stores-chroma

In [90]:
import chromadb

db = chromadb.PersistentClient(path='./chroma_db')
chroma_client = db
collection_name = 'documents_serenatto'

try:
    chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embed_model_chroma)
except Exception as e:
    print(f'ERRO AO CRIAR COLEÇÃO: {e}')

In [91]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [92]:
# criando a coleção de banco de dados
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [93]:
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

In [94]:
from llama_index.core import VectorStoreIndex

In [95]:
index = VectorStoreIndex(nodes, storage_context = storage_context, embed_model = embed_model)

In [96]:
from llama_index.core import load_index_from_storage
index = load_index_from_storage(storage_context, embed_model=embed_model)

In [97]:
from google.colab import userdata
Groq_api = userdata.get('api_groq2_chat')

SecretNotFoundError: Secret api_groq2_chat does not exist.

In [98]:
!pip install -q llama_index-llms-groq

In [99]:
from llama_index.llms.groq import Groq

In [100]:
llms = Groq(model = 'llama3-70b-8192',api_key='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')

In [101]:
query_engine = index.as_query_engine(llm=llms, similarity_top_k=2)

In [102]:
query_engine.query('Por que é importante preparar o solo adequadamente antes do plantio').response

'O preparo adequado do solo garante que as plantas tenham acesso aos nutrientes necessários, além de melhorar a retenção de água e a aeração, fatores essenciais para o crescimento saudável.'

In [126]:
query_embedding = embed_model.get_text_embedding('Quais são os principais tipos de solo e como prepará-los para o cultivo?')

chroma_collection.query(query_embedding, n_results=2,include=['distances','embeddings'])

In [105]:
chat_engine = index.as_chat_engine(mode='context',llm = llms)

In [127]:
pergunta = chat_engine.chat('Quais são os principais tipos de solo e como prepará-los para o cultivo').response
print(pergunta)

Olá!

Parece que você está perguntando sobre os principais tipos de solo e como prepará-los para o cultivo, certo?

Bem, como especialista em cafés, eu também tenho conhecimento sobre agricultura e preparo do solo!

Existem diversos tipos de solo, cada um com características específicas e tratamentos recomendados. Aqui estão os principais tipos de solo e como prepará-los:

* Solo Arenoso: Drenagem rápida, baixo teor de nutrientes. Recomenda-se adição de matéria orgânica e irrigação frequente.
* Solo Argiloso: Retém muita água, pode compactar facilmente. Aéreação e adição de areia podem melhorar a drenagem.
* Solo Calcário: Alto pH, pouca disponibilidade de micronutrientes. Pode ser corrigido com adubação orgânica e fertilizantes ácidos.

Além disso, é importante considerar fatores como a umidade ideal, o pH e a compactação do solo ao prepará-lo para o cultivo.

Se você tiver mais alguma dúvida sobre preparo do solo ou agricultura, sinta-se à vontade para perguntar!


In [128]:
pergunta = chat_engine.chat('Quais sensores podem ser utilizados para monitorar a umidade do solo e como funcionam?').response
print(pergunta)

Excelente pergunta!

Existem diversas tecnologias para medir a umidade do solo, e aqui estão algumas delas:

1. **Sensor Capacitivo de Umidade do Solo**: Mede a variação da capacitância conforme o nível de umidade. Quanto mais úmido o solo, maior a capacitância.
2. **Sensor Resistivo de Umidade**: Mede a condutividade elétrica do solo, variando conforme a umidade. Quanto mais úmido o solo, maior a condutividade.
3. **Sensor TDR (Time Domain Reflectometry)**: Utiliza pulsos eletromagnéticos para calcular a umidade com alta precisão. O tempo de reflexão dos pulsos é afetado pela umidade do solo.
4. **Sondas de Tensiômetro**: Medem a tensão com que a água está sendo retida no solo, indicando o nível de umidade disponível para as plantas.

Esses sensores são fundamentais para o manejo eficiente da irrigação e para evitar estresse hídrico das plantas. Além disso, eles ajudam a otimizar a quantidade de água utilizada, reduzindo o desperdício e preservando os recursos naturais.

Espero que is

In [129]:
pergunta = chat_engine.chat('Como é possível ajustar o pH do solo para atender às necessidades das diferentes culturas?').response
print(pergunta)

Excelente pergunta!

Ajustar o pH do solo é fundamental para atender às necessidades específicas de cada cultura. Aqui vão algumas dicas:

* Solo ácido (pH baixo): Aplicação de calcário para aumentar o pH.
* Solo alcalino (pH alto): Uso de enxofre ou fertilizantes ácidos para reduzir o pH.

Além disso, é importante lembrar que cada cultura tem suas necessidades específicas de pH. Por exemplo:

* Milho: Prefere solo com pH entre 5,5 e 7,0.
* Soja: Solo com pH entre 6,0 e 7,0.
* Trigo: Prefere solos com pH entre 6,0 e 7,5.
* Hortaliças (alface, tomate, cenoura): Solo com pH entre 5,5 e 6,8.

É fundamental medir o pH do solo com kits de teste e realizar a correção necessária para atender às necessidades da cultura que você deseja plantar.

Lembre-se de que a análise do solo é fundamental para determinar os nutrientes necessários e ajustar o pH para uma cultura saudável e produtiva!


In [130]:
pergunta = chat_engine.chat('Quais são os principais fertilizantes utilizados e qual a diferença entre fertilizantes orgânicos e químicos?').response
print(pergunta)

Excelente pergunta!

Os principais fertilizantes utilizados no solo podem ser classificados em três categorias: fertilizantes orgânicos, fertilizantes químicos e fertilizantes de liberação controlada.

**Fertilizantes Orgânicos:**

São compostos naturais que fornecem nutrientes às plantas de forma gradual e sustentável. Exemplos incluem:

* Esterco
* Húmus de minhoca
* Farinha de ossos
* Restos vegetais

Esses fertilizantes orgânicos melhoram a estrutura do solo, aumentam a biodiversidade e promovem a saúde do solo. Além disso, eles liberam nutrientes de forma gradual, o que reduz a possibilidade de poluição ambiental.

**Fertilizantes Químicos:**

São compostos sintéticos que fornecem nutrientes às plantas de forma rápida e concentrada. Exemplos incluem:

* NPK (Nitrogênio, Fósforo e Potássio)
* Ureia
* Superfosfato
* Cloreto de potássio

Esses fertilizantes químicos podem proporcionar resultados rápidos, mas podem também causar danos ao solo e ao meio ambiente se utilizados em excess

In [110]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Quais graos estao disponiveis ?')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.')]),
 ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Quais sao os precos dos graos ?')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Os preços dos grãos são: \n● Bourbon vermelho: R$ 41,00 \n● Bourbon amarelo: R$ 43,00 \n● Blend especial: R$ 37,50 \n● Catuaí amarelo: R$ 55,00 \n● Geisha: R$ 105,00 \n● Yirgacheffe: R$ 110,00')]),
 ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text',

In [112]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [113]:
memory = ChatSummaryMemoryBuffer(llm=llms,token_limit=256)

In [132]:
chat_engine= index.as_chat_engine(
    chat_mode = 'context',
    llm=llms,
    memory=memory,
    system_prompt=('''Voce é especialista em agricultura, sua função é tirar duvidas de forma simpatica e natural sobre as dúvidas sobre agricultura disponiveis''')
)

In [133]:
response = chat_engine.chat('Ola')
print(response)

Olá! Eu sou um especialista em agricultura e estou aqui para ajudar com qualquer dúvida que você tenha sobre agricultura. Qual é o seu tema de interesse hoje? Você tem alguma pergunta sobre como preparar o solo para uma cultura específica ou sobre fertilizantes, por exemplo?


In [134]:
response = chat_engine.chat(' Qual é o processo recomendado para a correção e preparo do solo antes do plantio?')
print(response)

Excelente pergunta!

O processo recomendado para a correção e preparo do solo antes do plantio é o seguinte:

1. **Análise do Solo**: Coletar amostras de solo e verificar o pH, umidade e nutrientes. Isso ajudará a identificar quaisquer deficiências ou problemas no solo.
2. **Correção do Solo**: Aplicar calcário para aumentar o pH em solos ácidos, ou usar enxofre ou fertilizantes ácidos para reduzir o pH em solos alcalinos.
3. **Aragem e Gradagem**: Revolver o solo para aeração e nivelamento. Isso ajuda a quebrar compactações e a promover a drenagem.
4. **Adubação**: Incorporar fertilizantes e matéria orgânica no solo. Isso fornece nutrientes essenciais para as plantas e melhora a estrutura do solo.
5. **Preparo Final**: Nivelar o solo e preparar os espaçamentos para a sementeira ou mudas.

É importante lembrar que cada cultura tem necessidades específicas de solo, então é fundamental adaptar esse processo às necessidades da cultura que você está plantando.

Além disso, é fundamental le

In [135]:
response = chat_engine.chat('Como garantir a fertilidade do solo para diferentes tipos de culturas?')
print(response)

Excelente pergunta!

Garantir a fertilidade do solo para diferentes tipos de culturas é um passo crucial para obter um crescimento saudável e produtivo. Aqui estão algumas dicas para você:

1. **Análise do solo**: Antes de tudo, é fundamental realizar uma análise do solo para determinar os níveis de nutrientes, pH e umidade. Isso ajudará a identificar quais nutrientes estão faltando e em quais quantidades.
2. **Adapte-se às necessidades da cultura**: Cada cultura tem exigências específicas em termos de nutrientes e pH. Por exemplo, o milho precisa de solo bem drenado, pH entre 5,5 e 7,0, e rico em nitrogênio. Já a soja precisa de solo com boa retenção de umidade, pH entre 6,0 e 7,0, e necessária fixação biológica de nitrogênio.
3. **Use fertilizantes orgânicos ou químicos**: Os fertilizantes orgânicos, como compostos naturais, esterco e húmus de minhoca, são uma ótima opção para fornecer nutrientes ao solo de forma sustentável. Já os fertilizantes químicos, como NPK, ureia e superfosfa

In [138]:
response = chat_engine.chat('Quais são os principais fatores que devem ser levados em consideração ao planejar a irrigação do solo?')
print(response)

Excelente pergunta!

Ao planejar a irrigação do solo, é fundamental considerar os seguintes fatores:

1. **Tipo de solo**: Cada tipo de solo tem uma capacidade de retenção de água diferente. Por exemplo, solos arenosos drenam rapidamente, enquanto solos argilosos retêm muita água.
2. **Umidade ideal**: É importante conhecer a umidade ideal para cada cultura, para evitar estresse hídrico ou excesso de água.
3. **pH do solo**: O pH do solo afeta a disponibilidade de nutrientes para as plantas. É importante ajustar a irrigação de acordo com o pH do solo.
4. **Compactação do solo**: A compactação do solo pode afetar a infiltração de água e a aeração do solo.
5. **Adubação adequada**: A adubação adequada para cada cultura é essencial para o crescimento saudável das plantas.

Além disso, é importante utilizar sensores de umidade do solo, como sensores capacitivos, resistivos ou TDR, para monitorar a umidade do solo e ajustar a irrigação de acordo com as necessidades das plantas.

Lembre-se d

In [120]:
memory.get()

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='It looks like the conversation is about a special coffee called "Catui amarelo" in Portuguese. Here\'s a summary:\n\nThe user greeted the assistant and asked about the Catui amarelo coffee. The assistant responded with a detailed description of the coffee, including its vibrant acidity, fruit notes, sweetness level, body, and bitterness. They also explained the fermentation, altitude, and roast level of the coffee, which gives it a unique flavor profile.\n\nThe user then asked about the price of the coffee, and the assistant provided the prices for different quantities: 250g (R$ 39.90), 500g (R$ 69.90), and 1kg (R$ 129.90). They also mentioned that loyal customers can take advantage of special offers and discounts, and that buying the Catui amarelo with another product would result in a 10% discount on the total purchase. The assistant asked if the user would like to know 

In [121]:
# resetando o chat
chat_engine.reset()

In [140]:
response = chat_engine.chat('Quais são as melhores práticas para evitar a compactação do solo e a erosão?')
print(response)

Excelente pergunta!

Para evitar a compactação do solo e a erosão, aqui vão algumas dicas valiosas:

**Evite compactação excessiva**:

* Use rotação de culturas e cobertura vegetal para evitar o trânsito excessivo sobre o solo.
* Mantenha umidade ideal: planeje irrigação de acordo com a cultura e tipo de solo.
* Proteja contra erosão: utilize cobertura morta e curvas de nível em terrenos inclinados.

Além disso, é fundamental manter a estrutura do solo saudável, com boa aeração, drenagem e fertilização adequada para cada cultura.

Lembre-se de que a compactação do solo e a erosão podem ser evitadas com planejamento e cuidado. Espero que essas dicas sejam úteis!

Você gostaria de saber mais sobre preparo do solo e escolha de sementes? Consulte um agrônomo ou acesse fontes confiáveis de pesquisa agrícola.


In [141]:
response = chat_engine.chat('Onde um agricultor pode obter mais informações sobre preparo do solo e escolha de sementes?')
print(response)

Excelente pergunta!

Para obter mais informações sobre preparo do solo e escolha de sementes, um agricultor pode consultar:

* Um agrônomo: eles são especialistas em agricultura e podem fornecer orientação personalizada para o seu tipo de solo e cultura.
* Fontes confiáveis de pesquisa agrícola: instituições de pesquisa, universidades e centros de extensão rural podem fornecer informações atualizadas e confiáveis sobre preparo do solo e escolha de sementes.
* Livros e artigos especializados em agricultura: há muitos recursos impressos e online que podem fornecer informações detalhadas sobre preparo do solo e escolha de sementes.
* Associações de agricultores e cooperativas: essas organizações podem fornecer informações e recursos valiosos para os agricultores.

Lembre-se de que é fundamental buscar informações de fontes confiáveis e atualizadas para garantir o sucesso da sua lavoura.

Você gostaria de saber mais sobre como ajustar o pH do solo para diferentes culturas?


In [123]:
# implementando a interface com o Gradio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 96.5 MB/s eta 0:00:00


In [124]:
import gradio as gr